In [28]:
import re
import datetime
import pandas

In [29]:
from IPython.display import display, HTML

In [30]:
results = pandas.read_csv('~/Downloads/draftkings-contest-entry-history.csv')

In [31]:
results['year'] = results['Contest_Date_EST'].apply(lambda k: k[0:4])
results['month'] = results['Contest_Date_EST'].apply(lambda k: k[0:7])
results['entry_fee'] = results['Entry_Fee'].apply(lambda k: float(re.sub('\$|,', '', k)))
results['recoup_nonticket'] = results['Winnings_Non_Ticket'].apply(lambda k: float(k.replace('$','').replace(',','')))
results['recoup_ticket']    = results['Winnings_Ticket'].apply(lambda k: float(k.replace('$','').replace(',','')))
results['recoup']           = results['recoup_nonticket'] + results['recoup_ticket']
results['pnl']              = results['recoup'] - results['entry_fee']

In [49]:
sport_years = results.groupby(['Sport','year','month'], as_index=False).aggregate({'entry_fee': 'sum','recoup': 'sum','pnl':'sum'})
sport_years.sort_values(['pnl',], ascending=False)

In [48]:
sport_years.groupby(['year', 'month'], as_index=False).aggregate({'pnl':'sum'})

In [34]:
nba = sport_years[sport_years['Sport']=='NBA']

In [47]:
nba

In [36]:
season_df = nba[nba['month'] > '2016-09']
season_fees = season_df['entry_fee'].sum()
season_recoup = season_df['recoup'].sum()
season_roi = (season_recoup - season_fees) / season_fees
season_roi

-0.12177769574560603

In [40]:
nba_contests = results[(results['Sport']=='NBA') & (results['month'] >= '2016-09')]

In [45]:
nba_slates = nba_contests.groupby('Contest_Date_EST', as_index=False).aggregate({'entry_fee': 'sum','recoup': 'sum','pnl':'sum'})

In [46]:
nba_slates['pnl'].sum()

-4618.4800000000005